In [1]:
# import os

# REPO_NAME = "Spatial_Equity_Analysis" 

# print("--- Enter notebook! ---")
# %cd {REPO_NAME}
# %cd notebooks

# print("--- Setup Complete! ---")

In [2]:
import geopandas as gpd
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import libpysal
import os

In [3]:
file_path = "../data/data_cleaned/"

graph_file_path = f"{file_path}singapore_walk_graph.graphml"
nodes_file_path  = f"{file_path}singapore_walk_graph_nodes.geojson"
edges_file_path  = f"{file_path}singapore_walk_graph_edges.geojson"

planning_area_file_path  = f"{file_path}planning_area.geojson"
sport_facilities_file_path  = f"{file_path}sport_facilities.geojson"
parks_file_path  = f"{file_path}parks.geojson"
park_connector_file_path  = f"{file_path}park_connector.geojson"
cycling_paths_file_path  = f"{file_path}cycling_paths.geojson"
hdb_file_path  = f"{file_path}hdb.geojson"

In [4]:
%ls -al

total 168
drwxr-xr-x 4 amber amber   4096 Dec 24 22:31 ./
drwxr-xr-x 6 amber amber   4096 Dec 23 02:56 ../
-rw-r--r-- 1 amber amber 125484 Dec 24 22:33 01_Data_Prep.ipynb
-rw-r--r-- 1 amber amber  26594 Dec 24 22:42 02_Network_Analysis.ipynb
drwxr-xr-x 6 amber amber   4096 Dec 24 22:32 Spatial_Equity_Analysis/
drwxr-xr-x 2 amber amber   4096 Dec 23 02:12 cache/


In [5]:
# CONSTANTS
CRS_PROJ = "EPSG:3414"  # Singapore SVY21
DISTANCE_THRESHOLD = 1600  # 1.5km max walking distance (~20 mins)

# Load pre-saved projected graph and GeoJSONs
G_proj = ox.load_graphml(graph_file_path)
G_nodes_proj = gpd.read_file(nodes_file_path)
G_edges_proj = gpd.read_file(edges_file_path)

# Project Graph to Meters only if needed
graph_crs = G_proj.graph.get("crs")
if graph_crs is None or str(graph_crs) != CRS_PROJ:
    G_proj = ox.project_graph(G_proj, to_crs=CRS_PROJ)

Skipping field highway: unsupported OGR type: 5
Skipping field lanes: unsupported OGR type: 5
Skipping field maxspeed: unsupported OGR type: 5
Skipping field name: unsupported OGR type: 5
Skipping field access: unsupported OGR type: 5
Skipping field service: unsupported OGR type: 5
Skipping field tunnel: unsupported OGR type: 5
Skipping field bridge: unsupported OGR type: 5
Skipping field width: unsupported OGR type: 5
Skipping field est_width: unsupported OGR type: 5


In [6]:
planning_area = gpd.read_file(planning_area_file_path )
sport_facilities  = gpd.read_file(sport_facilities_file_path)
hdb = gpd.read_file(hdb_file_path)
parks = gpd.read_file(parks_file_path)
park_connector = gpd.read_file(park_connector_file_path)
cycling_paths = gpd.read_file(cycling_paths_file_path)

In [7]:
planning_area = planning_area.to_crs(CRS_PROJ)
sport_facilities = sport_facilities.to_crs(CRS_PROJ)
park_connector = park_connector.to_crs(CRS_PROJ)
parks = parks.to_crs(CRS_PROJ)
cycling_paths = cycling_paths.to_crs(CRS_PROJ)
hdb = hdb.to_crs(CRS_PROJ)

G_nodes_proj = G_nodes_proj.to_crs(CRS_PROJ)
G_edges_proj = G_edges_proj.to_crs(CRS_PROJ)
edges = G_edges_proj.union_all()

In [8]:
def facility_preprocessing(gdf_facilities):
    """
    Prepares SportSG Facilities Data.
    """
    # Ensure CRS
    gdf_facilities = gdf_facilities.to_crs(CRS_PROJ)
    
    # Change the geometry from polygon Z to point (centroid )
    gdf_facilities['geometry'] = gdf_facilities.geometry.centroid
    
    # Assign ID
    gdf_facilities['facility_id'] = range(len(gdf_facilities))
    
    # Ensure we have a capacity column (Supply). 
    # Since floor area is not available, we'll assume 1
    if 'capacity' not in gdf_facilities.columns:
        gdf_facilities['capacity'] = 1
        
    return gdf_facilities

In [9]:
def population_preprocessing(gdf_pop):
    """
    Prepares HDB Population Data.
    """
    # Ensure CRS
    gdf_pop = gdf_pop.to_crs(CRS_PROJ)
    
    # Ensure we have centroids
    gdf_pop['centroid'] = gdf_pop.geometry.centroid
    
    # Assign ID
    gdf_pop['pop_id'] = range(len(gdf_pop))
    
    # Create a buffer for the "Euclidean Filter" step later
    gdf_pop['search_buffer'] = gdf_pop['centroid'].buffer(DISTANCE_THRESHOLD)
    
    return gdf_pop

In [10]:
def assign_network_nodes(gdf, G_proj, column_name):
    """
    Snaps points in a GeoDataFrame to the nearest nodes in the graph.
    Works for any point-based GDF (Population, Parks, Gyms).
    """
    # Ensure we are working with points (e.g. use centroids for polygons)
    geom = gdf.geometry.centroid if any(gdf.geom_type != 'Point') else gdf.geometry
    
    nodes = ox.nearest_nodes(G_proj, geom.x, geom.y)
    gdf[column_name] = nodes
    return gdf

In [11]:
def filter_pairs_by_buffer(gdf_demand, gdf_supply, 
                           demand_id='pop_id', supply_id='facility_id',
                           demand_node='origin_node', supply_node='dest_node',
                           demand_weight='population', supply_weight='capacity',
                           buffer_col='search_buffer'):
    """
    Performs a spatial join ensuring all necessary 2SFCA columns are preserved.
    """
    # 1. Prepare Demand GDF (use the buffer as the geometry)
    # We include ID, Node, and Weight (Population)
    demand_cols = [demand_id, demand_node, demand_weight, buffer_col]
    demand_prep = gpd.GeoDataFrame(gdf_demand[demand_cols], geometry=buffer_col, crs=gdf_demand.crs)
    
    # 2. Prepare Supply GDF
    # We include ID, Node, and Weight (Capacity)
    supply_cols = [supply_id, supply_node, supply_weight, 'geometry']
    supply_prep = gdf_supply[supply_cols]
    
    # 3. Join
    # 'inner' means we only keep demand-supply pairs that are within the Euclidean buffer
    joined = gpd.sjoin(
        supply_prep,
        demand_prep,
        how='inner',
        predicate='within'
    )
    
    return joined


In [12]:
def calculate_batch_distances(pairs_df, G_proj, 
                             demand_id='pop_id', supply_id='facility_id',
                             demand_node='origin_node', supply_node='dest_node',
                             demand_weight='population', supply_weight='capacity',
                             threshold=None):
    results = []
    cache = {}

    for _, row in pairs_df.iterrows():
        u, v = row[demand_node], row[supply_node]
        
        pair_key = (u, v)
        if pair_key in cache:
            dist = cache[pair_key]
        else:
            try:
                dist = nx.shortest_path_length(G_proj, u, v, weight='length')
                cache[pair_key] = dist
            except nx.NetworkXNoPath:
                cache[pair_key] = None
                continue
        
        if dist is not None and (threshold is None or dist <= threshold):
            results.append({
                'pop_id': row[demand_id],
                'facility_id': row[supply_id],
                'network_dist': dist,
                'population': row[demand_weight],
                'capacity': row[supply_weight]
            })
            
    return pd.DataFrame(results)

In [13]:
def calculate_m2sfca(df_od, distance_threshold, 
                     dist_col='network_dist', 
                     pop_col='population', 
                     supply_col='capacity'):
    """
    Calculate accessibility scores using the 2-Step Floating Catchment Area method
    with a Gaussian distance decay function.
    """
    df = df_od.copy()
    
    # --- Gaussian Decay Function ---
    # At 0m, weight = 1. At distance_threshold, weight approaches 0.01.
    v = -distance_threshold**2 / np.log(0.01) 
    df['weight'] = np.exp(-(df[dist_col])**2 / v)
    
    # --- Step 1: Supply-to-Demand Ratio (Rj) ---
    # Calculate weighted demand for each facility
    df['weighted_demand'] = df[pop_col] * df['weight']
    
    # Sum of weighted demand per Facility
    facility_demand_sum = df.groupby('facility_id')['weighted_demand'].sum().reset_index()
    facility_demand_sum.rename(columns={'weighted_demand': 'total_demand_at_j'}, inplace=True)
    
    # Merge back and calculate Ratio Rj = Supply / Sum(Weighted Demand)
    df = df.merge(facility_demand_sum, on='facility_id')
    df['Rj'] = df[supply_col] / df['total_demand_at_j']
    
    # --- Step 2: Accessibility Score (Ai) ---
    # How many "facility shares" are available to each resident?
    df['weighted_Rj'] = df['Rj'] * df['weight']
    
    # Sum weighted ratios per Population Point
    accessibility_scores = df.groupby('pop_id')['weighted_Rj'].sum().reset_index()
    accessibility_scores.rename(columns={'weighted_Rj': 'access_score'}, inplace=True)
    
    return accessibility_scores

In [14]:
# 1. Preprocess
hdb = population_preprocessing(hdb)
# sport_facilities = facility_preprocessing(sport_facilities)


In [15]:
hdb.head()

,name,population,geometry,centroid,pop_id,search_buffer
0,Path,242.446049,"LINESTRING (24481.043 47869.63, 24480.427 4786...",POINT (24457.667 47865.143),0,"POLYGON ((26057.667 47865.143, 26049.962 47708..."
1,Path,242.446049,"LINESTRING (18753.211 37893.756, 18764.786 379...",POINT (18772.678 37887.871),1,"POLYGON ((20372.678 37887.871, 20364.973 37731..."
2,Path,242.446049,"LINESTRING (41887.998 37282.122, 41889.532 372...",POINT (41907.212 37272.926),2,"POLYGON ((43507.212 37272.926, 43499.507 37116..."
3,Path,242.446049,"LINESTRING (18696.214 41806.326, 18691.537 418...",POINT (18723.768 41826.052),3,"POLYGON ((20323.768 41826.052, 20316.064 41669..."
4,Path,242.446049,"LINESTRING (26218.761 29036.217, 26222.34 2904...",POINT (26245.512 29051.375),4,"POLYGON ((27845.512 29051.375, 27837.808 28894..."


In [40]:
hdb.crs

<Projected CRS: EPSG:3414>
Name: SVY21 / Singapore TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Singapore - onshore and offshore.
- bounds: (103.59, 1.13, 104.07, 1.47)
Coordinate Operation:
- name: Singapore Transverse Mercator
- method: Transverse Mercator
Datum: SVY21
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
# sport_facilities.head()

In [17]:
# 3. Calculate Network Distances (The Slow Part)
# df_od_pairs = get_network_distances(hdb, sport_facilities, G_proj)


# 1. Assign Nodes
hdb = assign_network_nodes(hdb, G_proj, 'origin_node')
# sport_facilities = assign_network_nodes(sport_facilities, G_proj, 'dest_node')

# # 2. Pre-filter (The updated function now carries 'origin_node' through)
# pairs = filter_pairs_by_buffer(hdb, sport_facilities, demand_id='pop_id', supply_id='facility_id')

# # 3. Calculate Network Distances
# df_od_pairs = calculate_batch_distances( pairs, G_proj, threshold=DISTANCE_THRESHOLD)

# display(df_od_pairs.head())

In [18]:
# 4. Compute Score
# final_scores = calculate_m2sfca(df_od_pairs, DISTANCE_THRESHOLD)

In [19]:
# display(final_scores.head())

In [20]:
# # 5. Merge back to Map
# final_map = hdb.merge(final_scores, on='pop_id', how='left')

# # Fill NaN with 0 (areas with NO access)
# final_map['access_score'] = final_map['access_score'].fillna(0)

# # 6. Visualize
# final_map.plot(column='access_score', legend=True, cmap='viridis', figsize=(10, 10))

In [21]:
# # 7. Export GeoDataFrame to GeoJSON (WGS84)
# from pathlib import Path

# # Prepare a copy with only one geometry column
# final_map_export = final_map.copy()
# active_geom_name = final_map_export.geometry.name
# extra_geom_cols = [c for c, dt in final_map_export.dtypes.items() if dt == "geometry" and c != active_geom_name]
# if extra_geom_cols:
#     final_map_export = final_map_export.drop(columns=extra_geom_cols)

# # Reproject to WGS84 for GeoJSON/web mapping
# # GeoJSON (RFC 7946) requires WGS84 coordinates (EPSG:4326) in degrees; other CRSs are not allowed in the spec.
# # Web compatibility: Web map libraries (Leaflet, Mapbox GL, deck.gl) expect lon/lat GeoJSON; EPSG:3414 will render in the wrong place or be rejected
# final_map_wgs84 = final_map_export.to_crs("EPSG:4326")

# # Define output path
# output_path = "../data/data_cleaned/sportSG_2sfca.geojson"
# Path(Path(output_path).parent).mkdir(parents=True, exist_ok=True)

# # Save to GeoJSON
# final_map_wgs84.to_file(output_path, driver="GeoJSON")
# print(f"GeoJSON saved to: {output_path}")

In [22]:
parks.head()

,name,area,geometry
0,ANN SIANG HILL PK,3228.382276,"POLYGON ((29394.86 29306.735, 29398.146 29304...."
1,KRANJI MARSHES,567203.505838,"POLYGON ((16282.585 44868.334, 16182.492 44912..."
2,PASIR PANJANG PARK,59468.135315,"MULTIPOLYGON (((23398.012 28685.981, 23390.437..."
3,BOUGAINVILLEA PK,11333.268934,"POLYGON ((25237.943 34238.249, 25246.258 34239..."
4,TOA PAYOH TOWN PK,45692.560453,"POLYGON ((29550.33 34831.165, 29531.855 34833...."


In [55]:
from shapely import force_2d


def parks_preprocessing(gdf_parks, destination='Entrance', crs_proj=CRS_PROJ, edges=edges):
    # 1. Force lowercase for destination check to avoid string errors
    dest_type = str(destination).lower()
    
    # 2. Ensure CRS matches for both datasets
    gdf_parks = gdf_parks.to_crs(crs_proj)
    
    # Precompute centroids
    gdf_parks["centroid"] = gdf_parks.geometry.centroid
    gdf_parks = gdf_parks.reset_index(drop=True)
    gdf_parks["park_id"] = np.arange(len(gdf_parks))

    # Variables
    green_access = None
    entrance = []

    # Case 1: Use centroids directly
    if dest_type == 'centroid':
        ga = gdf_parks[["centroid", "area", "park_id"]].rename(columns={"centroid": "geometry"}).copy()
        green_access = gpd.GeoDataFrame(ga, geometry="geometry", crs=crs_proj)
    # Case 2: Derive entrances from boundary-edge intersections
    elif dest_type == 'entrance':
        # Buffer the park polygons slightly to capture nearby network edges
        gdf_parks["buffer"] = gdf_parks.geometry.buffer(20)
        # display(gdf_parks.head())
        for i, row in gdf_parks.iterrows():
            green_area = row['area']
            intersection = row['buffer'].boundary.intersection(edges)
            try:
                # Extract Points from whatever geometry was returned
                # (handles MultiPoint, Point)
                points = []
                
                if intersection.geom_type == 'Point':
                    points.append(intersection)
                elif intersection.geom_type == 'MultiPoint':
                    points.extend(list(intersection.geoms))

                for pt in points:
                    dic = {
                        'geometry': pt,
                        'area': green_area,
                        'park_id': i
                    }
                    entrance.append(dic)
            except: continue
        green_access = gpd.GeoDataFrame(entrance, crs=crs_proj)

    green_access['buffer'] = green_access['geometry'].buffer(25)
    w_e = libpysal.weights.fuzzy_contiguity(green_access['buffer'])
    green_access['entrance_components']= w_e.component_labels

    # Delete duplicate rows based on specific columns 
    green_access =  green_access.drop_duplicates(subset=["entrance_components"], keep='first')
    green_access = green_access.reset_index(drop=True)

    return green_access

In [56]:
green_access = parks_preprocessing(parks)
green_access.head()

/home/amber/miniconda3/envs/geopandas/lib/python3.12/site-packages/libpysal/weights/util.py:1618: UserWarning: The weights matrix is not fully connected: 
 There are 1882 disconnected components.
 There are 627 islands with ids: 4, 12, 15, 19, 20, 30, 35, 38, 107, 108, 109, 110, 111, 118, 136, 137, 173, 174, 175, 176, 183, 190, 196, 219, 246, 247, 253, 258, 267, 289, 307, 322, 325, 326, 327, 332, 334, 337, 338, 342, 347, 351, 353, 358, 393, 399, 400, 409, 415, 418, 419, 423, 444, 446, 452, 454, 456, 457, 480, 498, 502, 510, 512, 517, 520, 523, 524, 526, 528, 533, 535, 537, 539, 543, 544, 551, 557, 558, 563, 611, 654, 655, 667, 668, 672, 676, 682, 689, 734, 737, 743, 744, 747, 749, 751, 755, 758, 760, 762, 763, 765, 769, 786, 817, 821, 847, 854, 860, 912, 918, 920, 928, 935, 941, 946, 975, 980, 986, 988, 1008, 1009, 1012, 1013, 1016, 1061, 1063, 1068, 1073, 1075, 1080, 1092, 1108, 1112, 1114, 1119, 1124, 1125, 1130, 1132, 1133, 1134, 1135, 1143, 1144, 1148, 1151, 1153, 1165, 1166, 1168,

,geometry,area,park_id,buffer,entrance_components
0,POINT (29341.337 29330.682),3228.382276,0,"POLYGON ((29366.337 29330.682, 29366.217 29328...",0
1,POINT (29413.992 29229.415),3228.382276,0,"POLYGON ((29438.992 29229.415, 29438.872 29226...",1
2,POINT (29495.034 29178.834),3228.382276,0,"POLYGON ((29520.034 29178.834, 29519.913 29176...",2
3,POINT (29455.462 29308.61),3228.382276,0,"POLYGON ((29480.462 29308.61, 29480.342 29306....",3
4,POINT (15602.622 44514.158),567203.505838,1,"POLYGON ((15627.622 44514.158, 15627.501 44511...",4


In [57]:
# display(green_access)
display(green_access.crs)

<Projected CRS: EPSG:3414>
Name: SVY21 / Singapore TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Singapore - onshore and offshore.
- bounds: (103.59, 1.13, 104.07, 1.47)
Coordinate Operation:
- name: Singapore Transverse Mercator
- method: Transverse Mercator
Datum: SVY21
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [58]:
hdb

,name,population,geometry,centroid,pop_id,search_buffer,origin_node
0,Path,242.446049,"LINESTRING (24481.043 47869.63, 24480.427 4786...",POINT (24457.667 47865.143),0,"POLYGON ((26057.667 47865.143, 26049.962 47708...",6112962296
1,Path,242.446049,"LINESTRING (18753.211 37893.756, 18764.786 379...",POINT (18772.678 37887.871),1,"POLYGON ((20372.678 37887.871, 20364.973 37731...",1754637478
2,Path,242.446049,"LINESTRING (41887.998 37282.122, 41889.532 372...",POINT (41907.212 37272.926),2,"POLYGON ((43507.212 37272.926, 43499.507 37116...",3495933625
3,Path,242.446049,"LINESTRING (18696.214 41806.326, 18691.537 418...",POINT (18723.768 41826.052),3,"POLYGON ((20323.768 41826.052, 20316.064 41669...",2467386718
4,Path,242.446049,"LINESTRING (26218.761 29036.217, 26222.34 2904...",POINT (26245.512 29051.375),4,"POLYGON ((27845.512 29051.375, 27837.808 28894...",10761017333
...,...,...,...,...,...,...,...
13155,Path,242.446049,"LINESTRING (38562.747 34869.126, 38564.596 348...",POINT (38516.787 34855.615),13155,"POLYGON ((40116.787 34855.615, 40109.082 34698...",7402446510
13156,Path,242.446049,"LINESTRING (41987.505 37858.923, 41987.481 378...",POINT (41995.349 37879.878),13156,"POLYGON ((43595.349 37879.878, 43587.644 37723...",13390000434
13157,Path,242.446049,"LINESTRING (12010.493 36210.147, 12010.436 362...",POINT (12023.714 36219.604),13157,"POLYGON ((13623.714 36219.604, 13616.01 36062....",1131007637
13158,Path,242.446049,"LINESTRING (18646.526 41774.339, 18651.756 417...",POINT (18659.181 41759.974),13158,"POLYGON ((20259.181 41759.974, 20251.476 41603...",6656897566


In [59]:
green_access = assign_network_nodes(green_access, G_proj, 'dest_node')
pairs = filter_pairs_by_buffer(hdb, green_access, demand_id='pop_id', supply_id='park_id', supply_weight='area')

display(pairs)
display(pairs.info())

,park_id,dest_node,area,geometry,index_right,pop_id,origin_node,population
0,0,4636540811,3228.382276,POINT (29341.337 29330.682),28,28,249804231,242.446049
0,0,4636540811,3228.382276,POINT (29341.337 29330.682),29,29,3157401815,242.446049
0,0,4636540811,3228.382276,POINT (29341.337 29330.682),112,112,5209304187,242.446049
0,0,4636540811,3228.382276,POINT (29341.337 29330.682),253,253,7097151098,242.446049
0,0,4636540811,3228.382276,POINT (29341.337 29330.682),428,428,7097367263,242.446049
...,...,...,...,...,...,...,...,...
1881,447,425261703,2490.893050,POINT (30484.293 41350.554),5432,5432,13253850888,242.446049
1881,447,425261703,2490.893050,POINT (30484.293 41350.554),6003,6003,13253846455,242.446049
1881,447,425261703,2490.893050,POINT (30484.293 41350.554),7859,7859,8621167721,242.446049
1881,447,425261703,2490.893050,POINT (30484.293 41350.554),9757,9757,13253846459,242.446049


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 398231 entries, 0 to 1881
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   park_id      398231 non-null  int64   
 1   dest_node    398231 non-null  int64   
 2   area         398231 non-null  float64 
 3   geometry     398231 non-null  geometry
 4   index_right  398231 non-null  int64   
 5   pop_id       398231 non-null  int64   
 6   origin_node  398231 non-null  int64   
 7   population   398231 non-null  float64 
dtypes: float64(2), geometry(1), int64(5)
memory usage: 27.3 MB


None

In [60]:
def neareat_entrance(OD_mile): # OD_mile is the OD Matrix after selection, e.g real_distance < 1km
    unique = OD_mile['pop_index'].unique()
    OD_nearest_E = []
    for i in unique:
        df = OD_mile.loc[OD_mile['pop_index'] == i]
        df1 = df.groupby('park_id').min('distance')
        for j in df1.index:
            dic = {'pop_index': i,'park_id': j,'distance':df1.loc[j,'distance'],
              'green_area': df1.loc[j,'green_area'], 'pop_num':df1.loc[j,'pop_num']}
            OD_nearest_E.append(dic)
    OD_nearest_E = pd.DataFrame(OD_nearest_E)
    return OD_nearest_E

In [61]:
# df_od_pairs = calculate_batch_distances(pairs, G_proj, supply_id='park_id', supply_weight='area', threshold=DISTANCE_THRESHOLD)


In [62]:
# final_scores = calculate_m2sfca(df_od_pairs, DISTANCE_THRESHOLD)
# display(final_scores.head())

In [63]:
# final_map = hdb.merge(final_scores, on='pop_id', how='left')


In [64]:
# final_map['2sfca_score'] = final_map['2sfca_score'].fillna(0)
# final_map.plot(column='2sfca_score', legend=True, cmap='viridis', figsize=(10, 10))